In [3]:
import os
from pathlib import Path

In [4]:
print(Path().cwd())
current_dir = Path().cwd().parent
print(current_dir)

/workspaces/Kidney-Disease-Classification/research
/workspaces/Kidney-Disease-Classification


In [5]:
from dataclasses import dataclass

@dataclass
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augumentation: bool
    params_image_size: list

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import *

In [7]:
import tensorflow as tf

2026-01-11 09:10:53.763847: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-11 09:11:09.469530: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-11 09:11:20.280861: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
/workspaces/Kidney-Disease-Classification/.venv/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [10]:
current_dir, current_dir / CONFIG_FILE_PATH, current_dir / PARAMS_FILE_PATH

(PosixPath('/workspaces/Kidney-Disease-Classification'),
 PosixPath('/workspaces/Kidney-Disease-Classification/config/config.yaml'),
 PosixPath('/workspaces/Kidney-Disease-Classification/params.yaml'))

In [11]:
config = read_yaml(current_dir / CONFIG_FILE_PATH)
config

[2026-01-11 09:19:10,688: INFO: common: yaml file: /workspaces/Kidney-Disease-Classification/config/config.yaml loaded successfully]


ConfigBox({'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://drive.google.com/file/d/1gK9DTZ1ReF2a-LX1N0fEveYbQUTXDaHA/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/model_updated.h5'}, 'model_training': {'root_dir': 'artifacts/model_training', 'trained_model_path': 'artifacts/model_training/model.h5'}})

In [12]:
config.model_training

ConfigBox({'root_dir': 'artifacts/model_training', 'trained_model_path': 'artifacts/model_training/model.h5'})

In [14]:
params = read_yaml(current_dir / PARAMS_FILE_PATH)
params

[2026-01-11 09:21:40,186: INFO: common: yaml file: /workspaces/Kidney-Disease-Classification/params.yaml loaded successfully]


ConfigBox({'AUGUMENTATION': True, 'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 16, 'INCLUDE_TOP': False, 'EPOCHS': 1, 'CLASSES': 2, 'WEIGHTS': 'imagenet', 'LEARNING_RATE': 0.01})

In [16]:
config.data_ingestion.unzip_dir

'artifacts/data_ingestion'

In [20]:
data_path = current_dir / config.data_ingestion.unzip_dir / "kidney-ct-scan-image"
data_path.exists() , data_path

(True,
 PosixPath('/workspaces/Kidney-Disease-Classification/artifacts/data_ingestion/kidney-ct-scan-image'))

In [30]:
class ConfigurationManager:
    def __init__(self, config_filepath = current_dir / CONFIG_FILE_PATH, params_filepath = current_dir / PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        model_training = self.config.model_training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = current_dir / self.config.data_ingestion.unzip_dir / "kidney-ct-scan-image"
        create_directories([current_dir / model_training.root_dir])

        training_config = TrainingConfig(
            root_dir = current_dir / model_training.root_dir,
            trained_model_path = current_dir / Path(model_training.trained_model_path),
            updated_base_model_path = current_dir / Path(prepare_base_model.updated_base_model_path),
            training_data = training_data,
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augumentation = params.AUGUMENTATION,
            params_image_size = params.IMAGE_SIZE
        )

        return training_config

In [31]:
config_manager = ConfigurationManager()
training_config = config_manager.get_training_config()
training_config

[2026-01-11 09:52:21,782: INFO: common: yaml file: /workspaces/Kidney-Disease-Classification/config/config.yaml loaded successfully]
[2026-01-11 09:52:21,786: INFO: common: yaml file: /workspaces/Kidney-Disease-Classification/params.yaml loaded successfully]
[2026-01-11 09:52:21,787: INFO: common: created directory at: artifacts]
[2026-01-11 09:52:21,788: INFO: common: created directory at: /workspaces/Kidney-Disease-Classification/artifacts/model_training]


TrainingConfig(root_dir=PosixPath('/workspaces/Kidney-Disease-Classification/artifacts/model_training'), trained_model_path=PosixPath('/workspaces/Kidney-Disease-Classification/artifacts/model_training/model.h5'), updated_base_model_path=PosixPath('/workspaces/Kidney-Disease-Classification/artifacts/prepare_base_model/model_updated.h5'), training_data=PosixPath('/workspaces/Kidney-Disease-Classification/artifacts/data_ingestion/kidney-ct-scan-image'), params_epochs=1, params_batch_size=16, params_is_augumentation=True, params_image_size=BoxList([224, 224, 3]))

In [32]:
import urllib.request as request
from zipfile import ZipFile
import time

In [33]:
loaded_model = tf.keras.models.load_model(
    training_config.updated_base_model_path
)

2026-01-11 09:57:03.963514: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


[2026-01-11 09:57:07,774: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]


In [36]:
dict(
    a = 1./255
), 1/255

({'a': 0.00392156862745098}, 0.00392156862745098)

In [37]:
datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

datagenerator_kwargs

{'rescale': 0.00392156862745098, 'validation_split': 0.2}

In [39]:
dataflow_kwargs = dict(
            target_size=training_config.params_image_size[:-1],
            batch_size=training_config.params_batch_size,
            interpolation="bilinear"
        )
dataflow_kwargs

{'target_size': [224, 224], 'batch_size': 16, 'interpolation': 'bilinear'}

In [41]:
valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
valid_datagenerator

In [42]:
training_config.training_data

PosixPath('/workspaces/Kidney-Disease-Classification/artifacts/data_ingestion/kidney-ct-scan-image')

In [43]:
valid_generator = valid_datagenerator.flow_from_directory(
            directory=training_config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

Found 93 images belonging to 2 classes.


In [47]:
train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip = True,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                shear_range = 0.2,
                zoom_range = 0.2,
                **datagenerator_kwargs
            )
train_generator = train_datagenerator.flow_from_directory(
            directory=training_config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

Found 372 images belonging to 2 classes.


In [48]:
train_generator, valid_generator

(<keras.src.legacy.preprocessing.image.DirectoryIterator at 0x7585745eb5c0>,
 <keras.src.legacy.preprocessing.image.DirectoryIterator at 0x758574551f70>)

In [52]:
train_generator.samples, valid_generator.samples

(372, 93)

In [ ]:
print(f"93 * 4 = {93*4}")

93 * 4 = 372


In [53]:
train_generator.batch_size, valid_generator.batch_size

(16, 16)

In [59]:
372//16,  93//16

(23, 5)

In [ ]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        self.model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=["accuracy"]
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train_validation_generator(self):
        datagenerator_kargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        validation_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.validation_generator = validation_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )

        if self.config.params_is_augumentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip = True,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                shear_range = 0.2,
                zoom_range = 0.2,
                **datagenerator_kwargs
            )

        else:
            train_datagenerator = validation_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.validation_generator.samples // self.validation_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.validation_generator
        )

        self.save_model(
            path = self.config.trained_model_path,
            model = self.model
        )

In [81]:
config = ConfigurationManager()

[2026-01-11 11:42:10,031: INFO: common: yaml file: /workspaces/Kidney-Disease-Classification/config/config.yaml loaded successfully]
[2026-01-11 11:42:10,034: INFO: common: yaml file: /workspaces/Kidney-Disease-Classification/params.yaml loaded successfully]
[2026-01-11 11:42:10,034: INFO: common: created directory at: artifacts]


In [82]:
training_config = config.get_training_config()
training = Training(config = training_config)

[2026-01-11 11:42:12,410: INFO: common: created directory at: /workspaces/Kidney-Disease-Classification/artifacts/model_training]


In [83]:
training.get_base_model()

[2026-01-11 11:42:20,645: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]


In [84]:
training.train_validation_generator()

Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.


In [85]:
training.train()

2026-01-11 11:42:32.923016: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 205520896 exceeds 10% of free system memory.
2026-01-11 11:42:33.498644: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 205520896 exceeds 10% of free system memory.
2026-01-11 11:42:34.390435: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 51380224 exceeds 10% of free system memory.
2026-01-11 11:42:34.423352: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 102760448 exceeds 10% of free system memory.
2026-01-11 11:42:34.788508: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 102760448 exceeds 10% of free system memory.


23/23 ━━━━━━━━━━━━━━━━━━━━ 164s 7s/step - accuracy: 0.7219 - loss: 8.9191 - val_accuracy: 0.5375 - val_loss: 7.7379
[2026-01-11 11:45:27,774: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]


In [88]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_validation_generator()
    training.train()
    
except Exception as e:
    raise e

[2026-01-11 12:24:59,312: INFO: common: yaml file: /workspaces/Kidney-Disease-Classification/config/config.yaml loaded successfully]
[2026-01-11 12:24:59,316: INFO: common: yaml file: /workspaces/Kidney-Disease-Classification/params.yaml loaded successfully]
[2026-01-11 12:24:59,317: INFO: common: created directory at: artifacts]
[2026-01-11 12:24:59,317: INFO: common: created directory at: /workspaces/Kidney-Disease-Classification/artifacts/model_training]
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
23/23 ━━━━━━━━━━━━━━━━━━━━ 161s 7s/step - accuracy: 0.7500 - loss: 5.8573 - val_accuracy: 1.0000 - val_loss: 0.0054
